In [0]:
import mlflow
from mlflow.tracking import MlflowClient
import os
import shutil
import tempfile

#TODO : Config
CATALOG_NAME =  "tmp_cat_dev"
SCHEMA_NAME = "aranda_ml"
MODEL_NAME = "perceptron_classifier"

MODEL_FULL_NAME = f'{CATALOG_NAME}.{SCHEMA_NAME}.{MODEL_NAME}'

CATALOG_NAME_PRD = "tmp_cat_dev"
SCHEMA_NAME_PRD = "aranda"

TARGET_MODEL_FULL_NAME = f'{CATALOG_NAME_PRD}.{SCHEMA_NAME_PRD}.{MODEL_NAME}'

EXPERIMENT = "perceptron_classifier_experiment"
LOCAL_DIR = "/dbfs/FileStore/modelos"
UC = "databricks-uc"

mlflow.set_registry_uri(UC)

client = MlflowClient()
models = client.search_model_versions(filter_string=f"name='{MODEL_FULL_NAME}'")

best_model = None
best_accuracy_score = float('-inf')

for model in models:
    run_id = model.run_id
    run_data = client.get_run(run_id).data

    if "accuracy_score" in run_data.metrics:
        accuracy_score = run_data.metrics["accuracy_score"]
        if accuracy_score > best_accuracy_score:
            best_accuracy_score = accuracy_score
            best_model = model

if best_model:
    run_id = best_model.run_id
    run_info = client.get_run(run_id)

    new_model_version = mlflow.register_model(model_uri=f"models:/{MODEL_FULL_NAME}/{best_model.version}", name=TARGET_MODEL_FULL_NAME)
    client.set_registered_model_alias(TARGET_MODEL_FULL_NAME, "Champion", new_model_version.version)
   
    experiment_name = f"/{EXPERIMENT}"

    mlflow.set_experiment(experiment_name)
    
    if run_info.info.artifact_uri:
        artifact_uri = run_info.info.artifact_uri

        if not os.path.exists(LOCAL_DIR):
            os.mkdir(LOCAL_DIR)
        local_path = client.download_artifacts(run_id, EXPERIMENT, LOCAL_DIR)

        with mlflow.start_run():
            mlflow.log_param("source_model", MODEL_FULL_NAME)
            mlflow.log_param("target_model", TARGET_MODEL_FULL_NAME)
            mlflow.log_metric("accuracy_score", round(best_accuracy_score, 3))
            mlflow.log_artifacts(local_path)

        print(f"Experimento registrado en MLflow: {experiment_name}")
else:
    print("No se encuentra modelo.")
